In [1]:
! ~/.local/bin/aws s3 ls dev-rhassan/tony/

2020-04-10 17:14:45      62233 tmax001.png
2020-04-10 17:15:02      56764 tmax003.png
2020-04-10 17:15:10      55980 tmax004.png


In [2]:
! ~/.local/bin/aws s3 ls dev-rhassan/tony/ | wc

      3      12     129


In [3]:
! ~/.local/bin/aws s3 rm s3://dev-rhassan/tony/ --recursive

delete: s3://dev-rhassan/tony/tmax001.png
delete: s3://dev-rhassan/tony/tmax004.png
delete: s3://dev-rhassan/tony/tmax003.png


In [4]:
! ~/.local/bin/aws s3 ls dev-rhassan/tony/

In [5]:
! ~/.local/bin/aws s3 ls dev-rhassan/

                           PRE TRR_test_cases/
                           PRE africa-scene-li/
                           PRE africa-scene-lists/
                           PRE newsslcert/
2020-04-10 15:00:03        315 get-cw-log-stream
2020-04-10 13:15:07     276255 lpgs_integration_testing_test_case_512_TC-512-10000-list.txt
2020-04-01 19:06:46      89930 tc_512_cloud_end_to_end.pdf


In [6]:
from playLib.bo_bucket import bo_get_bucket_list
bucket_name = 'ga-et-data'
prefix='Cloud_Veg_ET/Data/TMAX/'
lista=bo_get_bucket_list(bucket_name, prefix)

List count = 366


In [7]:
'''
# This is serial and VERY SLOW!
import os

bucket='ga-et-data'
for file_obj in lista:
    
    prefix = '/'.join(file_obj.split('/')[0:-1]) + '/'
    #print(prefix)
    filen = file_obj.split('/')[-1]
    #print(filen)
    argv = ','.join([bucket,prefix,filen])
    print(argv)
    cmd = 'python3 ./png-maker/png-maker.py ' + argv
    os.system(cmd) 
'''

"\n# This is serial and VERY SLOW!\nimport os\n\nbucket='ga-et-data'\nfor file_obj in lista:\n    \n    prefix = '/'.join(file_obj.split('/')[0:-1]) + '/'\n    #print(prefix)\n    filen = file_obj.split('/')[-1]\n    #print(filen)\n    argv = ','.join([bucket,prefix,filen])\n    print(argv)\n    cmd = 'python3 ./png-maker/png-maker.py ' + argv\n    os.system(cmd) \n"

In [8]:
! ~/.local/bin/aws s3 ls dev-rhassan/tony/ | wc

      0       0       0


In [9]:
! ~/.local/bin/aws s3 ls dev-rhassan/tony/

# Part 2 Queues with RabittMQ

In [10]:
! sudo systemctl status rabbitmq

Unit rabbitmq.service could not be found.
>

In [11]:
! sudo systemctl --unit=list

systemctl: unrecognized option '--unit=list'


In [12]:
%%bash
#systemctl --type=service
systemctl status rabbitmq-server.service |grep Active


   Active: inactive (dead) since Fri 2020-04-10 16:08:00 UTC; 1h 8min ago


In [13]:
! (cd rabbit; docker-compose up -d)

rabbit_web_1 is up-to-date
rabbit_redis_1 is up-to-date
rabbit_tony_1 is up-to-date
rabbitmq is up-to-date


In [14]:
from playLib.qbird import Qbird

q = Qbird('localhost','tonyq')
q.q_declare()
msg='COVID-19 bites!!!!'
q.q_publish(msg)

In [15]:
for i in range(10):
    msg = 'COVID-19 bites!!!!' +str(i)
    q.q_publish(msg)

In [16]:
! docker ps

CONTAINER ID        IMAGE                          COMMAND                  CREATED             STATUS              PORTS                                                                                        NAMES
fd99d93bd65b        tbutzer/bash-tester            "docker-entrypoint.s…"   About an hour ago   Up 5 minutes        6379/tcp                                                                                     rabbit_tony_1
c5ecbbf6982b        tbutzer/stackdemo              "python myapp.py"        About an hour ago   Up 5 minutes        0.0.0.0:8000->8000/tcp                                                                       rabbit_web_1
4497bc911125        redis:alpine                   "docker-entrypoint.s…"   About an hour ago   Up 5 minutes        6379/tcp                                                                                     rabbit_redis_1
c8c69e577d14        rabbitmq:3-management-alpine   "docker-entrypoint.s…"   About an hour ago   Up 5 minutes        

In [32]:
! sudo docker exec rabbitmq rabbitmqctl status

Status of node rabbit@c8c69e577d14 ...
Runtime

OS PID: 164
OS: Linux
Uptime (seconds): 1103
RabbitMQ version: 3.8.3
Node name: rabbit@c8c69e577d14
Erlang configuration: Erlang/OTP 22 [erts-10.7.1] [source] [64-bit] [smp:2:2] [ds:2:2:10] [async-threads:64]
Erlang processes: 430 used, 1048576 limit
Scheduler run queue: 1
Cluster heartbeat timeout (net_ticktime): 60

Plugins

Enabled plugin file: /etc/rabbitmq/enabled_plugins
Enabled plugins:

 * rabbitmq_management
 * rabbitmq_web_dispatch
 * rabbitmq_management_agent
 * amqp_client
 * cowboy
 * cowlib

Data directory

Node data directory: /var/lib/rabbitmq/mnesia/rabbit@c8c69e577d14

Config files

 * /etc/rabbitmq/rabbitmq.conf

Log file(s)

 * <stdout>

Alarms

(none)

Memory

Calculation strategy: rss
Memory high watermark setting: 0.4 of available memory, computed to: 3.3124 gb
code: 0.0302 gb (29.7 %)
other_system: 0.024 gb (23.58 %)
other_proc: 0.0226 gb (22.22 %)
allocated_unused: 0.0176 gb (17.28 %)
other_ets: 0.0029 gb (2.84 %)

In [33]:
! sudo docker exec rabbitmq rabbitmqctl list_queues

Timeout: 60.0 seconds ...
Listing queues for vhost / ...
name	messages
tonyq	11


# Now Consume those Right here

In [34]:
def callback(ch, method, properties, body):
    print(" [x] Received %r" % body)

In [35]:
qr=Qbird('localhost','tonyq')
qr.q_set_callback(callback)


In [36]:
qr.q_consume()

 [x] Received b'COVID-19 bites!!!!'
 [x] Received b'COVID-19 bites!!!!0'
 [x] Received b'COVID-19 bites!!!!1'
 [x] Received b'COVID-19 bites!!!!2'
 [x] Received b'COVID-19 bites!!!!3'
 [x] Received b'COVID-19 bites!!!!4'
 [x] Received b'COVID-19 bites!!!!5'
 [x] Received b'COVID-19 bites!!!!6'
 [x] Received b'COVID-19 bites!!!!7'
 [x] Received b'COVID-19 bites!!!!8'
 [x] Received b'COVID-19 bites!!!!9'


KeyboardInterrupt: 

In [37]:
! sudo docker exec rabbitmq rabbitmqctl list_queues

Timeout: 60.0 seconds ...
Listing queues for vhost / ...
name	messages
tonyq	0


# Q them again

In [39]:
q = Qbird('localhost','tonyq')
q.q_declare()
for i in range(10):
    msg = 'COVID-19 STILL bites!!!!' +str(i)
    q.q_publish(msg)

In [40]:
! sudo docker exec rabbitmq rabbitmqctl list_queues

Timeout: 60.0 seconds ...
Listing queues for vhost / ...
name	messages
tonyq	10


# Drain these messages using the admin gui port 

http://localhost:15672/
        
## ssh tunnel via 03<tab> 15672 mhonk

In [41]:
! sudo docker exec rabbitmq rabbitmqctl list_queues

Timeout: 60.0 seconds ...
Listing queues for vhost / ...
name	messages
tonyq	4


In [42]:
! sudo docker exec rabbitmq rabbitmqctl list_queues

Timeout: 60.0 seconds ...
Listing queues for vhost / ...
name	messages
tonyq	0


In [44]:
! sudo netstat -anlp |grep docker-proxy

tcp        0      0 172.18.0.1:57806        172.18.0.5:15672        ESTABLISHED 20731/docker-proxy  
tcp6       0      0 :::8000                 :::*                    LISTEN      20626/docker-proxy  
tcp6       0      0 :::5672                 :::*                    LISTEN      20755/docker-proxy  
tcp6       0      0 :::15672                :::*                    LISTEN      20731/docker-proxy  
tcp6       0      0 127.0.0.1:5672          127.0.0.1:41152         FIN_WAIT2   20755/docker-proxy  
tcp6       0      0 127.0.0.1:15672         127.0.0.1:37634         ESTABLISHED 20731/docker-proxy  
tcp6       0      0 127.0.0.1:5672          127.0.0.1:41140         FIN_WAIT2   20755/docker-proxy  
